In [22]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [2]:
df = pd.read_excel("./User_Trung_Mai_01_Nov_23_04_Feb_24_Logged_Time.xls", sheet_name="Report")
df.head(3)

,Unnamed: 0,Issue,Key,Logged,01/Nov/23,02/Nov/23,03/Nov/23,04/Nov/23,05/Nov/23,06/Nov/23,...,26/Jan/24,27/Jan/24,28/Jan/24,29/Jan/24,30/Jan/24,31/Jan/24,01/Feb/24,02/Feb/24,03/Feb/24,04/Feb/24
0,NaN,"""Phone"" field is missing in the stored members...",YGMP1-362,0.75,NaN,NaN,NaN,NaN,NaN,NaN,...,0.5,NaN,NaN,0.25,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,[0.3] Deployemnt process.,YGMP1-426,0.25,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,[API] Add profile management for Volunteers,YGMP1-168,0.25,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df.drop(df.tail(1).index, inplace=True)
df.drop("Unnamed: 0", axis=1, inplace=True)
df = df.fillna(0)

In [5]:
print(f"Tổng số task đã thực hiện trong thời gian qua: {df.shape[0]}")

Tổng số task đã thực hiện trong thời gian qua: 219


In [6]:
billable_df = df[df["Key"].apply(lambda x: x.startswith("YGMP1"))]
ultilized_df = df[df["Key"].apply(lambda x: x.startswith("CGR"))]

In [7]:
print(f"Tổng số task billable: {billable_df.shape[0]}")
print(f"Tổng số task utilized: {billable_df.shape[0] + ultilized_df.shape[0]}")

Tổng số task billable: 89
Tổng số task utilized: 176


In [85]:
tasks_data = {"utilized": billable_df.shape[0] + ultilized_df.shape[0]}
tasks_data["non-utilized"] = len(df) - tasks_data["utilized"]
fig = go.Figure()
fig.add_trace(go.Bar(x=list(tasks_data.keys()), y=list(tasks_data.values()), text=list(tasks_data.values()), textposition='auto'))
fig.update_layout(
    title='Number of tasks',
    xaxis_title='Category',
    yaxis_title='Count',
    width=500,
)
fig.show()

In [84]:
tasks_data_pie = {"billable": billable_df.shape[0], "utilized": ultilized_df.shape[0]}
tasks_data_pie["non-utilized"] = len(df) - tasks_data_pie["utilized"] - tasks_data_pie["billable"]
fig = go.Figure()
fig.add_trace(go.Pie(labels=list(tasks_data_pie.keys()), values=list(tasks_data_pie.values()),
                     textinfo='label+percent', pull=[0.1, 0, 0]))  # Adjust the pull parameter as needed
fig.update_layout(
    title='Number of Tasks',
    width=500,
)
fig.show()

In [8]:
# sum(billable_df["Logged"]), sum(ultilized_df["Logged"])
print(f"Tổng thời gian log billable: {round(sum(billable_df['Logged']),2)} h")
print(f"Tổng thời gian log utilized: {round(sum(billable_df['Logged']) + sum(ultilized_df['Logged']),2)} h")

Tổng thời gian log billable: 177.32 h
Tổng thời gian log utilized: 380.23 h


In [86]:
hours_data = {"utilized": round(sum(billable_df['Logged']) + sum(ultilized_df['Logged']),2)}
hours_data["non-utilized"] = round(sum(df["Logged"]) - hours_data["utilized"], 2)
fig = go.Figure()
fig.add_trace(go.Bar(x=list(hours_data.keys()), y=list(hours_data.values()), text=list(hours_data.values()), textposition='auto'))
fig.update_layout(
    title='Hours logged',
    xaxis_title='Category',
    yaxis_title='Hours',
    width=500,
)
fig.show()

In [71]:
hours_data_pie = {"billable": round(sum(billable_df['Logged']),2), "utilized": round(sum(ultilized_df['Logged']),2)}
hours_data_pie["non-utilized"] = round(sum(df["Logged"]) - hours_data_pie["utilized"] - hours_data_pie["billable"], 2)
fig = go.Figure()
fig.add_trace(go.Pie(labels=list(hours_data_pie.keys()), values=list(hours_data_pie.values()),
                     textinfo='label+percent', pull=[0.1, 0, 0]))  # Adjust the pull parameter as needed
fig.update_layout(
    title='Hours logged',
    width=500,
)
fig.show()

In [75]:
df["Project"] = df["Key"].apply(lambda key: key[:3])
fig = go.Figure()
fig.add_trace(go.Bar(x=df["Project"].value_counts().index, y=df["Project"].value_counts().values, text=list(df["Project"].value_counts().values), textposition='outside'))
fig.update_layout(
    title='Project Distribution',
    xaxis_title='Project',
    yaxis_title='Hours',
    width=800,
)
fig.show()

In [67]:
fig = go.Figure()
fig.add_trace(go.Pie(labels=df["Project"].value_counts().index, values=df["Project"].value_counts().values,
                     textinfo='label+percent', pull=[0.1, 0, 0]))  # Adjust the pull parameter as needed
fig.update_layout(title='Project Distribution', width=800)
fig.show()

In [81]:
project_hours_data = []
for proj in df["Project"].value_counts().index:
    proj_df = df[df["Key"].apply(lambda x: x.startswith(proj))]
    project_hours_data.append((proj, round(sum(proj_df["Logged"]), 2)))

In [83]:
labels, values = zip(*project_hours_data)
fig = px.pie(names=labels, values=values, title='Project Hours Distribution')
fig.show()

In [10]:
# billable_df[billable_df["Logged"].apply(lambda x: x > 5)][["Key", "Logged"]]
filtered_df = billable_df[billable_df["Logged"].apply(lambda x: x > 5)][["Key", "Logged"]]

fig = px.bar(
    filtered_df,
    x="Key",
    y="Logged",
    text="Logged",
    title="Billable tasks with more than 5 hours logged",
    labels={"Logged": "Logged Hours", "Key": "Task"},
    height=500,
)
fig.update_traces(textposition='outside')
fig.update_layout(
    xaxis=dict(tickangle=45, tickmode='linear'),
    yaxis_title="Logged Hours",
    xaxis_title="Key",
)
fig.show()

In [11]:
# billable_df[billable_df["Logged"].apply(lambda x: x > 5)][["Key", "Logged"]]
filtered_df = billable_df[billable_df["Logged"].apply(lambda x: x < 1)][["Key", "Logged"]]

fig = px.bar(
    filtered_df,
    x="Key",
    y="Logged",
    text="Logged",
    title="Billable tasks less than 1 hour logged",
    labels={"Logged": "Logged Hours", "Key": "Task"},
    height=500,
)
fig.update_traces(textposition='outside')
fig.update_layout(
    xaxis=dict(tickangle=45, tickmode='linear'),
    yaxis_title="Logged Hours",
    xaxis_title="Key",
)
fig.show()

In [12]:
# ultilized_df[ultilized_df["Logged"].apply(lambda x: x > 5)][["Key", "Logged"]]
filtered_df = ultilized_df[ultilized_df["Logged"].apply(lambda x: x > 5)][["Key", "Logged"]]

fig = px.bar(
    filtered_df,
    x="Key",
    y="Logged",
    text="Logged",
    title="Utilized tasks with more than 5 hours logged",
    labels={"Logged": "Logged Hours", "Key": "Task"},
    height=500,
)
fig.update_traces(textposition='outside')
fig.update_layout(
    xaxis=dict(tickangle=45, tickmode='linear'),
    yaxis_title="Logged Hours",
    xaxis_title="Key",
)
fig.show()

In [13]:
filtered_df = ultilized_df[ultilized_df["Logged"].apply(lambda x: x < 1)][["Key", "Logged"]]

fig = px.bar(
    filtered_df,
    x="Key",
    y="Logged",
    text="Logged",
    title="Billable tasks less than 1 hour logged",
    labels={"Logged": "Logged Hours", "Key": "Task"},
    height=500,
)
fig.update_traces(textposition='outside')
fig.update_layout(
    xaxis=dict(tickangle=45, tickmode='linear'),
    yaxis_title="Logged Hours",
    xaxis_title="Key",
)
fig.show()

In [38]:
date_columns = [date for date in df.columns if date not in ["Issue", "Key", "Logged", "Project"]]

In [29]:
work_data = []
for col in date_columns:
    if sum(df[col]) > 8:
        work_data.append((col, sum(df[col])))
print(f"Số ngày làm việc hơn 8 tiếng: {len(work_data)} ngày")

Số ngày làm việc hơn 8 tiếng: 26 ngày


In [37]:
no_work_data = []
for col in date_columns:
    if sum(df[col]) < 8:
        no_work_data.append((col, sum(df[col])))
print(f"Số ngày nghỉ: {len(no_work_data)} ngày")

Số ngày nghỉ: 29 ngày


In [43]:
work_days = len(date_columns) - len(no_work_data)
print(f"Tổng số ngày đi làm: {work_days}")
print(f"Tỉ lệ số ngày làm hơn 8 tiếng: {round((len(work_data) / work_days) * 100, 2)}%")

Tổng số ngày đi làm: 67
Tỉ lệ số ngày làm hơn 8 tiếng: 38.81%


In [30]:
dates, values = zip(*work_data)
fig = go.Figure()
fig.add_trace(go.Bar(x=dates, y=values, text=values, textposition='outside'))
fig.update_layout(
    title='Days with more than 8 hours logged',
    xaxis_title='Date',
    yaxis_title='Value',
)
fig.show()

In [45]:
print(f"Trung bình log 1 ngày: {sum(df['Logged']) / work_days}")

Trung bình log 1 ngày: 8.20865671641791
